This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/trajopt.html).  I recommend having both windows open, side-by-side!


In [ ]:
import time

import numpy as np
import pydot
from IPython.display import SVG, clear_output, display
from pydrake.all import (
    Binding,
    Box,
    GraphOfConvexSets,
    GraphOfConvexSetsOptions,
    GurobiSolver,
    HPolyhedron,
    L2NormCost,
    MathematicalProgram,
    MosekSolver,
    Point,
    PointCloud,
    Rgba,
    RigidTransform,
    RotationMatrix,
    Solve,
    Sphere,
    StartMeshcat,
    eq,
)

from underactuated import running_as_notebook

In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

# Trajectory optimization using mixed-integer convex optimization

Note: These examples don't work yet with the open-source solvers.  One needs a Mosek/Gurobi license (See [this Drake tutorial](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/%2Flicensed_solvers_deepnote.ipynb) for setting up the license on Deepnote).

In [ ]:
mip_solver_available = (MosekSolver().available() and MosekSolver().enabled()) or (
    GurobiSolver().available() and GurobiSolver().enabled()
)

A_bl = np.array([[-1, 0], [0, -1], [1, 0], [0, 1], [1, 1]])
A_br = np.array([[1, 0], [0, -1], [-1, 0], [0, 1], [-1, 1]])
A_tl = np.array([[-1, 0], [0, 1], [1, 0], [0, -1], [1, -1]])
A_tr = np.array([[1, 0], [0, 1], [-1, 0], [0, -1], [-1, -1]])
b = np.array([4, 4, 0, 0, -1])

H_bl = HPolyhedron(A_bl, b)
H_br = HPolyhedron(A_br, b)
H_tl = HPolyhedron(A_tl, b)
H_tr = HPolyhedron(A_tr, b)


def plot_and_interact(meshcat, prog, x, start, goal):
    meshcat.Delete()
    meshcat.DeleteAddedControls()
    meshcat.Set2dRenderMode(xmin=-4, xmax=4, ymin=-4, ymax=4)
    meshcat.SetObject("obstacle", Box(np.sqrt(2), 1, np.sqrt(2)), Rgba(1, 0, 0, 1))
    meshcat.SetTransform(
        "obstacle",
        RigidTransform(RotationMatrix.MakeYRotation(np.pi / 4), [0, 2, 0]),
    )

    meshcat.SetObject("start", Sphere(0.1), Rgba(0, 0, 1, 1))
    meshcat.SetObject("goal", Sphere(0.1), Rgba(0, 1, 0, 1))

    x_start = start.evaluator().lower_bound()
    x_goal = goal.evaluator().lower_bound()
    x_sol = np.zeros((x.shape[0], 3))
    meshcat.AddSlider(
        "x_start",
        -4,
        4,
        0.25,
        x_start[0],
        decrement_keycode="ArrowLeft",
        increment_keycode="ArrowRight",
    )
    meshcat.AddSlider(
        "y_start",
        -4,
        -1,
        0.25,
        x_start[1],
        decrement_keycode="ArrowDown",
        increment_keycode="ArrowUp",
    )
    meshcat.AddSlider("x_goal", -4, 4, 0.25, x_goal[0])
    meshcat.AddSlider("y_goal", 1, 4, 0.25, x_goal[1])
    pts = PointCloud(x.shape[0])
    pts.mutable_xyzs()[1, :] = 0

    if not running_as_notebook:
        return

    meshcat.AddButton("Stop")
    while meshcat.GetButtonClicks("Stop") < 1:
        x_start = [
            meshcat.GetSliderValue("x_start"),
            meshcat.GetSliderValue("y_start"),
        ]
        x_goal = [
            meshcat.GetSliderValue("x_goal"),
            meshcat.GetSliderValue("y_goal"),
        ]
        start.evaluator().UpdateLowerBound(x_start)
        start.evaluator().UpdateUpperBound(x_start)
        goal.evaluator().UpdateLowerBound(x_goal)
        goal.evaluator().UpdateUpperBound(x_goal)

        result = Solve(prog)
        clear_output(wait=True)
        print(f"Feasible: {result.is_success()}")
        print(f"Cost: {result.get_optimal_cost()}")

        x_sol[:, [0, 2]] = result.GetSolution(x)

        meshcat.SetLine("x", x_sol.T, 2.0, Rgba(0, 0, 1, 1))
        meshcat.SetTransform("start", RigidTransform([x_start[0], 0, x_start[1]]))
        meshcat.SetTransform("goal", RigidTransform([x_goal[0], 0, x_goal[1]]))
        pts.mutable_xyzs()[0, :] = x_sol[:, 0]
        pts.mutable_xyzs()[2, :] = x_sol[:, 2]
        meshcat.SetObject("x_pts", pts, 5.0, rgba=Rgba(0, 0, 1, 1))

        time.sleep(0.1)

    meshcat.DeleteAddedControls()


def plot_and_interact_spp(meshcat, G, source, target, relaxation):
    meshcat.Delete()
    meshcat.DeleteAddedControls()
    meshcat.Set2dRenderMode(xmin=-4, xmax=4, ymin=-4, ymax=4)
    meshcat.SetObject("obstacle", Box(np.sqrt(2), 1, np.sqrt(2)), Rgba(1, 0, 0, 1))
    meshcat.SetTransform(
        "obstacle",
        RigidTransform(RotationMatrix.MakeYRotation(np.pi / 4), [0, 2, 0]),
    )

    meshcat.SetObject("start", Sphere(0.1), Rgba(0, 0, 1, 1))
    meshcat.SetObject("goal", Sphere(0.1), Rgba(0, 1, 0, 1))

    x_start = source.set().x()
    x_goal = target.set().x()
    np.zeros((x_start.shape[0], 3))
    meshcat.AddSlider(
        "x_start",
        -4,
        4,
        0.25,
        x_start[0],
        decrement_keycode="ArrowLeft",
        increment_keycode="ArrowRight",
    )
    meshcat.AddSlider(
        "y_start",
        -4,
        -1,
        0.25,
        x_start[1],
        decrement_keycode="ArrowDown",
        increment_keycode="ArrowUp",
    )
    meshcat.AddSlider("x_goal", -4, 4, 0.25, x_goal[0])
    meshcat.AddSlider("y_goal", 1, 4, 0.25, x_goal[1])
    pts = PointCloud(2)
    pts.mutable_xyzs()[1, :] = 0

    if not relaxation and not mip_solver_available:
        return

    if not running_as_notebook:
        return

    meshcat.AddButton("Stop")
    while meshcat.GetButtonClicks("Stop") < 1:
        x_start = [
            meshcat.GetSliderValue("x_start"),
            meshcat.GetSliderValue("y_start"),
        ]
        x_goal = [
            meshcat.GetSliderValue("x_goal"),
            meshcat.GetSliderValue("y_goal"),
        ]
        source.set().set_x(x_start)
        target.set().set_x(x_goal)

        options = GraphOfConvexSetsOptions()
        options.convex_relaxation = relaxation
        result = G.SolveShortestPath(source, target, options)
        clear_output(wait=True)
        print(f"Feasible: {result.is_success()}")
        print(f"Cost: {result.get_optimal_cost()}")

        for e in G.Edges():
            u = result.GetSolution(e.xu())
            v = result.GetSolution(e.xv())
            phi = result.GetSolution(e.phi())
            meshcat.SetLineSegments(
                f"edges/{e.id().get_value()}",
                [u[0], 0, u[1]],
                [v[0], 0, v[1]],
                2.0,
                Rgba(0, 0, 1, 1),
            )
            meshcat.SetProperty(f"edges/{e.id().get_value()}", "visible", phi > 0.5)
            pts.mutable_xyzs()[[0, 2], 0] = u
            pts.mutable_xyzs()[[0, 2], 1] = v
            meshcat.SetObject(
                f"edges/{e.id().get_value()}/pts",
                pts,
                5.0,
                rgba=Rgba(0, 0, 1, 1),
            )

        meshcat.SetTransform("start", RigidTransform([x_start[0], 0, x_start[1]]))
        meshcat.SetTransform("goal", RigidTransform([x_goal[0], 0, x_goal[1]]))

        time.sleep(0.1)

    meshcat.DeleteAddedControls()

## Non-convex collision constraints

To form the baseline for our comparison, let's first take the existing tools and apply them here.

In [ ]:
def nonconvex_example():
    meshcat.Delete()
    N = 10  # number of time steps
    x_start = np.array([0.2, -2.0])
    x_goal = np.array([0, 2.0])

    prog = MathematicalProgram()

    x = prog.NewContinuousVariables(N, 2, "x")  # positions

    # Initial and final value constraints:
    start = prog.AddBoundingBoxConstraint(x_start, x_start, x[0])
    goal = prog.AddBoundingBoxConstraint(x_goal, x_goal, x[N - 1])

    def vertex_collision(x):
        # |x[0]| + |x[1]| is not differentiable at the origin, so we use an
        # alternative formulation:
        return [np.maximum((x[0] + x[1]) ** 2, (x[0] - x[1]) ** 2)]

    def edge_collision(x):
        a = x[:2]
        b = x[2:]
        # Write the nonconvex disjunctive constraint
        orthant = [
            np.minimum(a[0] + a[1], b[0] + b[1]),
            np.minimum(a[0] - a[1], b[0] - b[1]),
            np.minimum(-a[0] - a[1], -b[0] - b[1]),
            np.minimum(-a[0] + a[1], -b[0] + b[1]),
        ]
        return [np.max(orthant)]

    for n in range(N - 1):
        # Stage cost (sum of squared distances):
        # prog.AddCost(dist, np.concatenate([x[n],x[n+1]]))
        step = x[n + 1] - x[n]
        prog.AddQuadraticCost(step.dot(step))

        # Collision avoidance constraints |x|_1 >= 1
        # prog.AddConstraint(vertex_collision, [1], [np.inf], x[n])
        prog.AddConstraint(
            edge_collision, [1], [np.inf], np.concatenate([x[n], x[n + 1]])
        )

    prog.SetInitialGuess(
        x, np.linspace(x_start, x_goal, N) + 0.1 * np.random.rand(N, 2)
    )

    plot_and_interact(meshcat, prog, x, start, goal)


nonconvex_example()

## NOTE: Mixed-integer Programming (MIP) Solvers

Drake wraps a number of open-source and commercial solvers.  The open-source solvers are sufficient for many problems, but they do not support mixed-integer optimization. To run the MIP versions of the examples below (e.g. `convex_relaxation=False`), please follow the instructions [here](https://drake.mit.edu/bazel.html#proprietary-solvers) if you're running on your local machine or [here](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/licensed_solvers_deepnote-0f645b52d59340ca9e9e0e7280c371ef) if you are running on Deepnote.

## Big-M formulation (binaries per face)

Note: If we wanted to minimize distance (instead of sum of squared distances), we could use, e.g.
```
  prog.AddLorentzConeConstraint(np.array([Expression(l2norm[0,n]), step[0], step[1]]))
  prog.AddLinearCost(l2norm[0,n])
```

In [ ]:
def bigM_example(convex_relaxation=False):
    if not convex_relaxation and not mip_solver_available:
        print(
            "You need an MIP solver to run this example (but you can still run the convex relaxation). See the instructions above."
        )
        return

    N = 10  # number of time steps
    bigM = 10
    x_start = np.array([0.0, -2.0])
    x_goal = np.array([0.0, 2.0])

    prog = MathematicalProgram()

    x = prog.NewContinuousVariables(N, 2, "x")  # positions
    if convex_relaxation:
        b = prog.NewContinuousVariables(N - 1, 4, "b")  # binaries for collisions
        prog.AddBoundingBoxConstraint(0, 1, b)
    else:
        b = prog.NewBinaryVariables(N - 1, 4, "b")  # binaries for collisions

    # Initial and final value constraints:
    start = prog.AddBoundingBoxConstraint(x_start, x_start, x[0])
    goal = prog.AddBoundingBoxConstraint(x_goal, x_goal, x[N - 1])

    for n in range(N - 1):
        # Stage cost (sum of squared distances encourages equal-sized steps):
        step = x[n + 1] - x[n]
        prog.AddQuadraticCost(step.dot(step))

        # Collision avoidance constraints (each *line segment* must be in a region)
        for k in [n, n + 1]:
            prog.AddLinearConstraint(
                x[k].dot([1, 1]) <= -1 + (1 - b[n, 0]) * bigM
            )  # bottom left
            prog.AddLinearConstraint(
                x[k].dot([-1, 1]) <= -1 + (1 - b[n, 1]) * bigM
            )  # bottom right
            prog.AddLinearConstraint(
                x[k].dot([1, -1]) <= -1 + (1 - b[n, 2]) * bigM
            )  # top left
            prog.AddLinearConstraint(
                x[k].dot([-1, -1]) <= -1 + (1 - b[n, 3]) * bigM
            )  # top right
        prog.AddLinearConstraint(np.sum(b[n]) == 1)

    plot_and_interact(meshcat, prog, x, start, goal)


bigM_example(convex_relaxation=False)

In [ ]:
bigM_example(convex_relaxation=True)

## Convex hull (binaries per region) using perspective functions

In [ ]:
def convex_hull_example(convex_relaxation=False):
    if not convex_relaxation and not mip_solver_available:
        print(
            "You need an MIP solver to run this example (but you can still run the convex relaxation). See the instructions above."
        )
        return

    N = 10  # number of time steps
    x_start = np.array([0.0, -2.0])
    x_goal = np.array([0.0, 2.0])

    Region = [H_bl, H_br, H_tl, H_tr]

    prog = MathematicalProgram()

    x = prog.NewContinuousVariables(N, 2, "x")  # positions
    if convex_relaxation:
        b = prog.NewContinuousVariables(N - 1, 4, "b")  # binaries for regions
        prog.AddBoundingBoxConstraint(0, 1, b)
    else:
        b = prog.NewBinaryVariables(N - 1, 4, "b")  # binaries for regions

    # Slack variables for the regions
    x_region = [
        prog.NewContinuousVariables(N, 2, "x_bl"),
        prog.NewContinuousVariables(N, 2, "x_br"),
        prog.NewContinuousVariables(N, 2, "x_tl"),
        prog.NewContinuousVariables(N, 2, "x_tr"),
    ]
    for n in range(N):
        prog.AddLinearConstraint(eq(x[n], sum([xr[n] for xr in x_region])))

    # Initial and final value constraints:
    start = prog.AddLinearConstraint(eq(x[0], x_start))
    goal = prog.AddLinearConstraint(eq(x[N - 1], x_goal))

    for n in range(N - 1):
        # Stage cost (sum of squared distances encourages equal-sized steps)
        step = x[n + 1] - x[n]
        prog.AddQuadraticCost(step.dot(step))

        # Region containment constraints
        for i in range(4):
            Region[i].AddPointInNonnegativeScalingConstraints(
                prog, x_region[i][n], b[n, i]
            )
        prog.AddLinearConstraint(np.sum(b[n]) == 1)

    plot_and_interact(meshcat, prog, x, start, goal)


convex_hull_example()

To avoid cutting the corners, we can define regions for each line segment (using the Cartesian power of the set).

In [ ]:
def convex_hull_example2(convex_relaxation=False):
    if not convex_relaxation and not mip_solver_available:
        print(
            "You need an MIP solver to run this example (but you can still run the convex relaxation). See the instructions above."
        )
        return

    N = 10  # number of time steps
    x_start = np.array([0.0, -2.0])
    x_goal = np.array([0.0, 2.0])

    Region = [H_bl, H_br, H_tl, H_tr]
    # Each *line segment* must be in a region.
    Region2 = [R.CartesianPower(2) for R in Region]

    prog = MathematicalProgram()

    x = prog.NewContinuousVariables(N, 2, "x")  # positions
    if convex_relaxation:
        b = prog.NewContinuousVariables(N - 1, 4, "b")  # binaries for regions
        prog.AddBoundingBoxConstraint(0, 1, b)
    else:
        b = prog.NewBinaryVariables(N - 1, 4, "b")  # binaries for regions

    # Slack variables for the regions
    x_region = [
        prog.NewContinuousVariables(N - 1, 4, "xA"),
        prog.NewContinuousVariables(N - 1, 4, "xB"),
        prog.NewContinuousVariables(N - 1, 4, "xC"),
        prog.NewContinuousVariables(N - 1, 4, "xD"),
    ]
    for n in range(N - 1):
        prog.AddLinearConstraint(eq(x[n], sum([xr[n, :2] for xr in x_region])))
        prog.AddLinearConstraint(eq(x[n + 1], sum([xr[n, 2:] for xr in x_region])))

    # Initial and final value constraints:
    start = prog.AddLinearConstraint(eq(x[0], x_start))
    goal = prog.AddLinearConstraint(eq(x[N - 1], x_goal))

    for n in range(N - 1):
        # Stage cost (sum of squared distances encourages equal-sized steps)
        step = x[n + 1] - x[n]
        prog.AddQuadraticCost(step.dot(step))

        # Region containment constraints
        for i in range(4):
            Region2[i].AddPointInNonnegativeScalingConstraints(
                prog, x_region[i][n], b[n, i]
            )
        prog.AddLinearConstraint(np.sum(b[n]) == 1)

    plot_and_interact(meshcat, prog, x, start, goal)


convex_hull_example2()

TODO: Make the point that the squared-norm is an odd cost function (for instance, changing the number of samples N changes the cost), but using the L2 norm leads to strange behavior in the formulations above.  It indicates that there is something a little wrong with the formulation.  GCS resolves it...

## Graph of Convex Sets

In [ ]:
def graph_of_convex_sets_example(convex_relaxation=False, show_graph=False):
    if not convex_relaxation and not mip_solver_available:
        print(
            "You need an MIP solver to run this example (but you can still run the convex relaxation). See the instructions above."
        )
        return

    b = np.array([-1, -1, 4])
    A_left = np.array([[1, 1], [1, -1], [-1, 0]])
    A_right = np.array([[-1, -1], [-1, 1], [1, 0]])
    A_top = np.array([[-1, -1], [1, -1], [0, 1]])
    A_bottom = np.array([[1, 1], [-1, 1], [0, -1]])

    H_left = HPolyhedron(A_left, b)
    H_right = HPolyhedron(A_right, b)
    H_top = HPolyhedron(A_top, b)
    H_bottom = HPolyhedron(A_bottom, b)

    G = GraphOfConvexSets()
    source = G.AddVertex(Point([0.2, -2]), "source")
    left = G.AddVertex(H_left, "left")
    right = G.AddVertex(H_right, "right")
    top = G.AddVertex(H_top, "top")
    bottom = G.AddVertex(H_bottom, "bottom")
    target = G.AddVertex(Point([0, 2]), "target")

    E = [
        [source, left],
        [source, bottom],
        [source, right],
        [bottom, left],
        [bottom, right],
        [left, top],
        [right, top],
        [left, target],
        [top, target],
        [right, target],
    ]

    # TODO: Support creating L2NormCosts from symbolic
    # (e.g. np.linalg.norm(e.xu() - e.xv()))
    cost = L2NormCost(A=np.block([np.eye(2), -np.eye(2)]), b=np.zeros((2, 1)))
    for u, v in E:
        e = G.AddEdge(u, v)
        e.AddCost(Binding[L2NormCost](cost, np.concatenate([e.xu(), e.xv()])))

    if show_graph:
        options = GraphOfConvexSetsOptions()
        options.convex_relaxation = convex_relaxation
        result = G.SolveShortestPath(source, target, options)
        display(
            SVG(
                pydot.graph_from_dot_data(
                    G.GetGraphvizString(result, show_slacks=True)
                )[0].create_svg()
            )
        )
    else:
        plot_and_interact_spp(meshcat, G, source, target, convex_relaxation)


graph_of_convex_sets_example(convex_relaxation=True)

In [ ]:
graph_of_convex_sets_example(show_graph=True)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=05031f8c-3586-4b47-be79-7f4893cf2f9d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>